In [1]:
import numpy as np
import queue

### 1. Goal detection

In [2]:
# Check if goal has been satisfied for a given design in the queue
def check_goal(layout, target_gate_count):
    gate_count = np.count_nonzero(layout == 2)
    if gate_count == target_gate_count:
        return True
    else:
        return False

### 2. Candidate representation and children generation

In [3]:
# Generates all potential children for current design that satisfy both rules for all land spaces
def generate_children(parent):
    children_list = []    
    
    # Find dimensions of parent design
    height = np.shape(parent)[0]
    width = np.shape(parent)[1]
    
    for i in range(height):
        for j in range(width):
            
            # Check for empty land area
            if parent[i,j] == 0:
                
                new_potential_walkway_child = np.copy(parent)
                new_potential_terminal_child = np.copy(parent)
                
                # Update empty land with terminal and add to children list if it passes both rules
                new_potential_terminal_child[i,j] = 2
                
                if check_child(new_potential_terminal_child):
                    children_list.append(new_potential_terminal_child)
                    
                # Update empty land with walkway and add to children list if it passes both rules
                new_potential_walkway_child[i,j] = 1                
                
                if check_child(new_potential_walkway_child):
                    children_list.append(new_potential_walkway_child)    
                
                
    return children_list

### Checks both rules for each potential design

In [4]:
def check_child(proposed_layout):
    
    # Checks for 3 bordering empty spaces and 1 bordering walkway
    def check_rule_1(design, x, y):
        N = design[x - 1, y]
        S = design[x + 1, y]
        E = design[x, y + 1]
        W = design[x, y - 1]

        current_location_NSEW_values = np.array([N, S, E, W])
        empty_NSEW_borders = np.count_nonzero(current_location_NSEW_values == 0)
        walkway_NSEW_borders = np.count_nonzero(current_location_NSEW_values == 1)

        if empty_NSEW_borders == 3 and walkway_NSEW_borders == 1:
            return True
        else:
            return False
        
    
    # Checks for a nearby walkway 
    def check_rule_2(design, x, y):
        N = design[x - 1, y]
        S = design[x + 1, y]
        E = design[x, y + 1]
        W = design[x, y - 1]

        NW = design[x - 1, y - 1]
        NE = design[x - 1, y + 1]
        SW = design[x + 1, y - 1]
        SE = design[x + 1, y + 1]
        
        current_location_NSEW_values = np.array([N, S, E, W])
        current_location_border_values = np.array([N, S, E, W, NW, NE, SW, SE])
        
        walkway_all_borders_count = np.count_nonzero(current_location_border_values == 1)
        terminal_NSEW_borders_count = np.count_nonzero(current_location_NSEW_values == 2)

        if walkway_all_borders_count > 0:
            return True
        
        # Also checks for a bordering terminal if there are no other walkways nearby
        # (this only is relevant for children of the seed so that the first walkway does not need
        # a second walkway to also have a terminal attached to it)
        elif (terminal_NSEW_borders_count >= 1 and walkway_all_borders_count == 0):
            return True
        else:
            return False
    
    height = np.shape(proposed_layout)[0]
    width = np.shape(proposed_layout)[1]
    
    
    for i in range(height):
        for j in range(width):
            
            # Initialize rule booleans
            rule_bool_1 = True
            rule_bool_2 = True
            
            # Check rule 1 if a terminal is found
            if proposed_layout[i,j] == 2:
                rule_bool_1 = check_rule_1(proposed_layout, i, j)
            
            # Check rule 2 is a walkway is found
            elif proposed_layout[i,j] == 1:
                rule_bool_2 = check_rule_2(proposed_layout, i, j)
                
            # Returns first false at first rule break
            if not (rule_bool_1 and rule_bool_2):
                return False    
    
    # Returns that the design is valid only if both rules have worked for all walkways and terminals
    return True

### 3. Code for depth first search

In [5]:
def DFS(seed_name, target_gate_count, depth):
    seed = np.genfromtxt(seed_name + '.csv', delimiter=',', dtype=int)
    print('Seed:\n', seed)
    
    # Generate initial empty stack (also known as a LIFO queue)
    airport_stack = queue.LifoQueue()
    
    # Initialize depth
    depth_0 = 0
    depthmax = depth

    # Attach depth level to seed
    airport_item = (depth_0, seed)
    
    # Add seed to stack
    airport_stack.put(airport_item)
    
    # Keep searching while stack is not empty.....
    while not airport_stack.empty():
        
        sol_bool = False
        # Grab the top of the stack
        stack_top = airport_stack.get()
    
        # Check if top of stack is the goal, return layout if true
        if check_goal(layout=stack_top[1], target_gate_count=target_gate_count):
            print('\n Solution found! \n')
            print(stack_top[1])
            print('Depth Level: {}'.format(stack_top[0]))
            sol_bool = True
            break
    
        # Create children if seed is not the goal and additional depth is possible
        if stack_top[0] < depthmax:
            current_children = generate_children(stack_top[1])
            current_children_count = len(current_children)        
            depth_child = stack_top[0] + 1

        # Add each design to stack along with depth level
            for child_number in range(current_children_count):
                airport_design = np.copy(current_children[child_number])
                airport_item = (depth_child, airport_design)
                airport_stack.put(airport_item, block=False)
    
    # Return no solution if stack is empty
    if not sol_bool:
        print('\n No solution for this seed')

### Results for depth first search

In [ ]:
A_star(seed_name='A', target_gate_count=3)

In [ ]:
A_star(seed_name='B', target_gate_count=4)

### 4. Transition function $g(n)$ and heuristic $h(n)$

In [15]:
# Heuristics for defining single child's objective function value
def evaluate_child(layout, target_gate_count):
    walkway_count = np.count_nonzero(layout == 1)
    gate_count = np.count_nonzero(layout == 2)
    
    g_n = walkway_count
    h_n = (target_gate_count - gate_count)
    f = g_n + h_n
    return f

### 5. Code for A*

In [11]:
def A_star(seed_name, target_gate_count):
    seed = np.genfromtxt(seed_name + '.csv', delimiter=',', dtype=int)
    print('Seed:\n', seed)
    eval_counter = 0
    
    # Generate initial empty queue
    airport_queue = queue.PriorityQueue()
    
    # Evaluate objective function score of seed
    seed_score = evaluate_child(layout=seed, target_gate_count=target_gate_count)

    # Attach score to seed
    airport_item = (seed_score, eval_counter, seed)
    
    # Add seed to priority queue
    airport_queue.put(airport_item)
    
    # Keep searching while queue is not empty.....
    while not airport_queue.empty():
    
        # Grab the top of the queue
        queue_top = airport_queue.get()        
    
        # Check if top of queue is the goal, return layout if true
        if check_goal(layout=queue_top[2], target_gate_count=target_gate_count):
            walkway_count = np.count_nonzero(queue_top[2] == 1)
            print('\n Solution found with {} walkways after {} evaluations! \n'.format(walkway_count, eval_counter))
            print(queue_top[2])
            return
    
        # Create children if seed is not the goal
        current_children = generate_children(queue_top[3])
        current_children_count = len(current_children)

        # Evaluate each design and add it to the priority queue
        for child_number in range(current_children_count):
            
            airport_design = np.copy(current_children[child_number])
            score = evaluate_child(layout=airport_design, target_gate_count=target_gate_count)
            eval_counter = eval_counter + 1
            
            airport_item = (score_1, score_2, eval_counter, airport_design)
            airport_queue.put(item=airport_item)
    
    # Return no solution if queue is empty
    print('\n No solution for this seed. Tested {} designs'.format(eval_counter))

### Results for A*

In [16]:
A_star(seed_name='A', target_gate_count=3)

Seed:
 [[-1 -1 -1 -1 -1]
 [-1  1  0  0 -1]
 [-1  0  0  0 -1]
 [-1  0  0  0 -1]
 [-1  0  0  0 -1]
 [-1  0  0  0 -1]
 [-1  0  0  0 -1]
 [-1  0  0  0 -1]
 [-1 -1 -1 -1 -1]]

 Solution found with 6 walkways after 9713 evaluations! 

[[-1 -1 -1 -1 -1]
 [-1  1  0  0 -1]
 [-1  1  2  0 -1]
 [-1  1  0  0 -1]
 [-1  1  2  0 -1]
 [-1  1  0  0 -1]
 [-1  1  2  0 -1]
 [-1  0  0  0 -1]
 [-1 -1 -1 -1 -1]]


In [17]:
A_star(seed_name='B', target_gate_count=4)

Seed:
 [[-1 -1 -1 -1 -1 -1 -1]
 [-1  0  0  0  0  0 -1]
 [-1  0  0  0  0  0 -1]
 [-1  1  0  0  0  0 -1]
 [-1  0  0  0  0  0 -1]
 [-1  0  0  0  0  0 -1]
 [-1 -1 -1 -1 -1 -1 -1]]

 Solution found with 4 walkways after 1818 evaluations! 

[[-1 -1 -1 -1 -1 -1 -1]
 [-1  0  0  0  0  0 -1]
 [-1  0  2  0  2  0 -1]
 [-1  1  1  1  1  0 -1]
 [-1  0  2  0  2  0 -1]
 [-1  0  0  0  0  0 -1]
 [-1 -1 -1 -1 -1 -1 -1]]


In [18]:
A_star(seed_name='C', target_gate_count=6)

Seed:
 [[-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1  0  0  0  0  0  0  0 -1]
 [-1  0  0  0  0  0  0  0 -1]
 [-1  0  0  0  1  0  0  0 -1]
 [-1  0  0  0  0  0  0  0 -1]
 [-1  0  0  0  0  0  0  0 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]]

 Solution found with 3 walkways after 6617 evaluations! 

[[-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1  0  0  0  0  0  0  0 -1]
 [-1  0  0  2  0  2  0  0 -1]
 [-1  0  2  1  1  1  2  0 -1]
 [-1  0  0  2  0  2  0  0 -1]
 [-1  0  0  0  0  0  0  0 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]]


In [ ]:
A_star(seed_name='D', target_gate_count=7)

Seed:
 [[-1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1  0  0  0  0  0  0  0  0 -1]
 [-1  0  0  0  0  0  0  0  0 -1]
 [-1  0  0  0  0  0  0  0  0 -1]
 [-1  0  0  0  0  0  0  0  0 -1]
 [-1  0  0  0  0  0  0  0  0 -1]
 [-1  0  0  0  0  0  0  0  0 -1]
 [-1  0  0  0  0  0  0  0  0 -1]
 [-1  0  0  0  0  1  0  0  0 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1]]


In [12]:
A_star(seed_name='E', target_gate_count=8)

Seed:
 [[-1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1  1  0  0 -1 -1 -1 -1 -1 -1]
 [-1  0  0  0  0 -1 -1 -1 -1 -1]
 [-1  0  0  0  0  0 -1 -1 -1 -1]
 [-1  0  0  0  0  0  0 -1 -1 -1]
 [-1  0  0  0  0  0  0  0 -1 -1]
 [-1  0  0  0  0  0  0  0  0 -1]
 [-1  0  0  0  0  0  0  0  0 -1]
 [-1  0  0  0  0  0  0  0  0 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1]]

 Solution found with 10 walkways after 22254 evaluations! 

[[-1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1  1  0  0 -1 -1 -1 -1 -1 -1]
 [-1  1  0  2  0 -1 -1 -1 -1 -1]
 [-1  0  1  1  0  0 -1 -1 -1 -1]
 [-1  0  2  0  1  2  0 -1 -1 -1]
 [-1  0  0  2  1  0  2  0 -1 -1]
 [-1  0  0  0  1  1  1  2  0 -1]
 [-1  0  2  1  0  2  0  0  0 -1]
 [-1  0  0  0  0  0  0  0  0 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1]]


In [ ]:
A_star(seed_name='F', target_gate_count=11)

In [32]:
A_star(seed_name='G', target_gate_count=4)

Seed:
 [[-1 -1 -1 -1 -1 -1 -1]
 [-1  0  0  0  0  0 -1]
 [-1  0  0  0  0  0 -1]
 [-1  0  0  1  0  0 -1]
 [-1  0  0  0  0  0 -1]
 [-1  0  0  0  0  0 -1]
 [-1 -1 -1 -1 -1 -1 -1]]

 Solution found with 1 walkways after 27 evaluations! 

[[-1 -1 -1 -1 -1 -1 -1]
 [-1  0  0  0  0  0 -1]
 [-1  0  0  2  0  0 -1]
 [-1  0  2  1  2  0 -1]
 [-1  0  0  2  0  0 -1]
 [-1  0  0  0  0  0 -1]
 [-1 -1 -1 -1 -1 -1 -1]]


### 6. Discussion